In [1]:
import requests
import json

In [2]:
osmose_url= "http://osmose.openstreetmap.fr/en/api/0.3beta/issues?osm_type=relation&osm_id={}&full=true"

In [3]:
bbox_for_overpass = "5.173011321882331,-4.258575439453126,5.544913134097361,-3.572616577148438"

In [4]:
overpass_base_url = "http://overpass-api.de/api/interpreter?data="
overpass_query_part_for_bus_lines = '[out:json];relation["route"~"bus|ferry"]({});relation["type"="route_master"](br);out meta;'.format(bbox_for_overpass)
overpass_query_part_for_bus_routes = '[out:json];relation["route"~"bus|ferry"]({});out meta;'.format(bbox_for_overpass)

bus_lines_call = requests.get(overpass_base_url + overpass_query_part_for_bus_lines)
if bus_lines_call.status_code != 200:
    print("erreur overpass :{}".format(bus_lines_call.status_code))

bus_routes_call = requests.get(overpass_base_url + overpass_query_part_for_bus_routes)
if bus_routes_call.status_code != 200:
    print("erreur overpass :{}".format(bus_routes_call.status_code))

bus_routes = bus_routes_call.json()["elements"]
bus_lines = bus_lines_call.json()["elements"]
bus_relations = [line['id'] for line in bus_lines] + [route['id'] for route in bus_routes]

In [5]:
len(bus_relations)

642

In [6]:
seems_ok = []
to_check = []

for a_relation in bus_relations:
    osmose_ = osmose_url.format(a_relation)
    osmose_call = requests.get(osmose_)
    osmose_results = osmose_call.json()['issues']
    if not osmose_results:
        seems_ok.append(a_relation)
    elif len(osmose_results) == 1:
        if osmose_results[0]['class'] == 21404 or "Duplicated similar values interval:conditional" in osmose_results[0]['subtitle']: #ignore "missing ref tag"
            seems_ok.append(a_relation)
        else :
            to_check.append(a_relation)
    else :
        to_check.append(a_relation)


In [7]:
seems_ok_objects = ["r{}".format(relation) for relation in seems_ok]
josm_ok = "http://localhost:8111/load_object?new_layer=true&relation_members=true&objects="+",".join(seems_ok_objects)
josm_ok

'http://localhost:8111/load_object?new_layer=true&relation_members=true&objects=r5985016,r10087961,r10173635,r10179006,r10179435,r10184139,r10184730,r10184964,r10185142,r10185169,r10185403,r10185406,r10187611,r10187867,r10189319,r10190159,r10190354,r10191924,r10192143,r10200077,r10201145,r10206110,r10206969,r10206986,r10211372,r10211655,r10211824,r10212284,r10213167,r10215245,r10215413,r10215677,r10216195,r10216288,r10220180,r10220245,r10220807,r10225666,r10228020,r10228047,r10228286,r10228332,r10228640,r10228832,r10228894,r10228996,r10229775,r10230294,r10231465,r10232988,r10233569,r10234049,r10235280,r10235433,r10237485,r10238035,r10240686,r10240809,r10241119,r10241191,r10241381,r10241430,r10241609,r10241995,r10246195,r10246667,r10246746,r10246874,r10247373,r10247522,r10247975,r10248235,r10256167,r10256311,r10257679,r10260743,r10261222,r10261637,r10265827,r10265924,r10266646,r10266864,r10267036,r10270241,r10270789,r10270989,r10271088,r10271257,r10279858,r10280602,r10281561,r10282981,r

In [8]:
to_check_objects = ["r{}".format(relation) for relation in to_check]
to_check_ok = "http://localhost:8111/load_object?new_layer=true&relation_members=true&objects="+",".join(to_check_objects)
to_check_ok

'http://localhost:8111/load_object?new_layer=true&relation_members=true&objects=r9589447,r10192153,r10199868,r10212293,r10212448,r10213431,r10413162,r10434093,r10178669,r10179434,r10220806,r10246092,r10247520,r10256955,r10283600,r10289988,r10291017,r10408692,r10413160,r10413161,r10434091,r10459358,r10474250,r10474251'

In [9]:
len(to_check_objects)

24